# Introduction

### what we will cover

### how we validate

### how we measure improvements

flops

### run commands and flags?



# Stupid implementation


### python

```python
start = monotonic()
for i in range(N):
    for j in range(N):
        for k in range(N):
            C[i][j] += A[i][k] * B[k][j]

```

average gflops: 0.015153
N: 256


### cpp

```cpp
void gemm_stupid(){
    for (int i = 0; i < N; i++)
        for (int j = 0; j < N; j++) {
            for (int k = 0; k < N; k++)
                C[i][j] += A[i][k] * B[k][j];
        }
}
```

average gflops: 3.738789
N: 256




# numpy (single threaded)

We use the library threadpoolctl to force numpy into only using a single thread as shown below.

```python
from threadpoolctl import threadpool_limits

with threadpool_limits(limits=1):
    pass
```

We use numpys `@`operator to do the matrix multiplication

```python
def gemm():
    return A @ B
```

average gflops: 43.073460
N: 2048

# numpy (multi threaded)

numpy by default is multithreaded so we will be testing out how good default numpy is.

We get an average gflops of 138, which is impressive but numpy can do much better.
Something very important to note is the way we generate the initial matrcies,
more importantly their byte sizes.




In [ ]:
import numpy as np

A = np.random.random((N, N))
B = np.random.random((N, N))

print(A.dtype)

: 

From this we can observe that numpys default array type is float64,
wich means that we can save up on alot of memory by converging these to float32,
squizzing numpy to its best performence.

We do the same test as above but with

```python
A = np.random.random((N, N)).astype(np.float32)
B = np.random.random((N, N)).astype(np.float32)
```

This gives us an impressive average of 390.94 - impressive!

We will be using this number as reference since c++ float is float32 by default (other wise double)

We will circle back to the cpp implemetation and try to achieve the high preformace of numpy,
and even beat it.

# Transposing 


```cpp


```

- why transposing will help with memory

- our findings




# 2d vs 1D arrays

# Blocking

## why does blocking even help? (images and memory)

### theoretical best based on cache size

## testing best 

We tested the variante that has **threading** and **transposing**
- Threading
- Transposed

| blockSize                              | Matrix Size | FLOP/s
|------------------------------------------|-------------|----------------
| 4                                 | 1024        | 101.024773 GFLOPS
| 8                                 | 1024        | 63.231953 GFLOPS
| 16                                | 2048        | 194.324829 GFLOPS
| 32                                | 1024        | 94.407333 GFLOPS



# Threading with OMP

# Unrolling

Unrolling is very nice idea that will help us remove the inner most loop by unrolling it.

- doing the if statement logic (compiler wise) is expensive


our currentl inner loop looks like this
```cpp
..
for (j = 0; j < blockSize; j++) 
    for (k = 0; k < blockSize; k++){
        C[bi + i][bj + j] += A[bi + i][bk + k] * B_trans[bj + j][bk + k];
    }
..

```

We have an issue where the inner most if loop is being updated for each entry of `k`
and this is expensive since the compiler needs to do extra work, and extra memory usage is done here to fetch and upate `k`

we can reduce this workload by unrolling `x` amount of instruciont and writing them down manually.

for example unrolling by 4 instrucionts would look like this

```cpp
..
for (j = 0; j < blockSize; j++) 
    for (k = 0; k < blockSize; k+4){
        C[bi + i][bj + j] += A[bi + i][bk + 0] * B_trans[bj + j][bk + 0];
        C[bi + i][bj + j] += A[bi + i][bk + 1] * B_trans[bj + j][bk + 1];
        C[bi + i][bj + j] += A[bi + i][bk + 2] * B_trans[bj + j][bk + 2];
        C[bi + i][bj + j] += A[bi + i][bk + 3] * B_trans[bj + j][bk + 3];
    }
..

```

However, our testing shows that this barely gives any performace advantage. Neither did unrolling 8 instrctions.

The problem is that we are still using the if statement, so the best scenario would be to unroll all the instrucions and remove the if statement.

the innest most loop runs up to blocksize, meaning there would be the same amount of instrcionts as the block size, so we need to unroll 16 instrucions
to be able to remove the if statement

```cpp
..
for (j = 0; j < blockSize; j++) {
        C[bi + i][bj + j] += A[bi + i][bk + k] * B_trans[bj + j][bk + k];
        C[bi + i][bj + j] += A[bi + i][bk + k + 1] * B_trans[bj + j][bk + k + 1];
        C[bi + i][bj + j] += A[bi + i][bk + k + 2] * B_trans[bj + j][bk + k + 2];
        C[bi + i][bj + j] += A[bi + i][bk + k + 3] * B_trans[bj + j][bk + k + 3];

        C[bi + i][bj + j] += A[bi + i][bk + k + 4] * B_trans[bj + j][bk + k + 4];
        C[bi + i][bj + j] += A[bi + i][bk + k + 5] * B_trans[bj + j][bk + k + 5];
        C[bi + i][bj + j] += A[bi + i][bk + k + 6] * B_trans[bj + j][bk + k + 6];
        C[bi + i][bj + j] += A[bi + i][bk + k + 7] * B_trans[bj + j][bk + k + 7];

        C[bi + i][bj + j] += A[bi + i][bk + k + 8] * B_trans[bj + j][bk + k + 8];
        C[bi + i][bj + j] += A[bi + i][bk + k + 9] * B_trans[bj + j][bk + k + 9];
        C[bi + i][bj + j] += A[bi + i][bk + k + 10] * B_trans[bj + j][bk + k + 10];
        C[bi + i][bj + j] += A[bi + i][bk + k + 11] * B_trans[bj + j][bk + k + 11];

        C[bi + i][bj + j] += A[bi + i][bk + k + 12] * B_trans[bj + j][bk + k + 12];
        C[bi + i][bj + j] += A[bi + i][bk + k + 13] * B_trans[bj + j][bk + k + 13];
        C[bi + i][bj + j] += A[bi + i][bk + k + 14] * B_trans[bj + j][bk + k + 14];
        C[bi + i][bj + j] += A[bi + i][bk + k + 15] * B_trans[bj + j][bk + k + 15];
    }
..

```
Suprising, this also gave us very little performace advantage, so we played aroudn with the block size and found that a blocksize of 4 (corresponding to 4 unrooled instrctions) gives the best performace

Final loop:

```cpp
..
for (j = 0; j < blockSize; j++) {
    // Fully unrolled for blockSize = 4
    C[bi + i][bj + j] += A[bi + i][bk + 0] * B_trans[bj + j][bk + 0];
    C[bi + i][bj + j] += A[bi + i][bk + 1] * B_trans[bj + j][bk + 1];
    C[bi + i][bj + j] += A[bi + i][bk + 2] * B_trans[bj + j][bk + 2];
    C[bi + i][bj + j] += A[bi + i][bk + 3] * B_trans[bj + j][bk + 3];
}
..

```

average gflops: 299.380005
N = 2048

Very good but not as good as numpty 32byte arrays.

# Apple accelerate

# Assembly level instructions (AMX, NEON, gcc)




| Description                              | Matrix Size | FLOP/s
|------------------------------------------|-------------|----------------
| Python without numpy                     | 256         | 0.0092 GFLOPS 
| st_np_matmul.py                          | 512         | 15.374713 GFLOPS
| st_PTB64.cpp                             | 1024        | 17.178770 GFLOPS
| np_matmul.py                             | 1024         | 52.210830 GFLOPS
| matrix_mult.cpp                          | 1024        | 34.398815 GFLOPS
| st_PTB1D.cpp                             | 1024        | 31.425362 GFLOPS
| st_PTB16.cpp                               | 1024        | 40.412571 GFLOPS
| PTB1D.cpp                                | 1024        | 127.062523 GFLOPS
| PTB4.cpp                                 | 1024        | 101.024773 GFLOPS
| PTB8.cpp                                 | 1024        | 63.231953 GFLOPS
| PTB16.cpp                                | 2048        | 194.324829 GFLOPS
| PTB32.cpp                                | 1024        | 94.407333 GFLOPS
| PTBunroll.cpp                                | 2048        | GFLOPS: 296.469116
 

| accelerate.cpp                           | 8192        | 1326.189209 GFLOPS


(blocksize over 16 make yields less FLOPs)
(-O3 is faster than -O2 for me)


Run the best code
`clang++ PTB.cpp -O3 -march=native -ffast-math -Xpreprocessor -fopenmp -I/opt/homebrew/include -L/opt/homebrew/lib -lomp -o compiled/PTB && ./compiled/PTB`


Accelerate

`clang++ -O3 -DACCELRATE_NEW_LAPACK -DACCELERATE_LAPACK_ILP64 accelerate.cpp -framework Accelerate -o ./compiled/accelerate && ./compiled/accelerate`


### Questions

Blocking helps with cache, but why? same with unrolling 

#pragma omp parallel for private(bj, bk, i, j, k) shared(A, B_trans, C) - what does this do *exacly*? why is bi not included?

In [ ]:
### Sources

Blocking
- https://gist.github.com/metallurgix/8ee5262ed818730b5cc0



George Hotz
- (add yt video link)


Miduem Article
- https://vaibhaw-vipul.medium.com/matrix-multiplication-optimizing-the-code-from-6-hours-to-1-sec-70889d33dcfa


Neon Arm Instructions
- https://developer.arm.com/architectures/instruction-sets/intrinsics/#f:@navigationhierarchiessimdisa=[Neon]&f:@navigationhierarchiesreturnbasetype=[float]&f:@navigationhierarchieselementbitsize=[32]&q=vld1


Apple Accelerate
- https://developer.apple.com/documentation/accelerate/vdsp-library


- AMX
- https://zhen8838.github.io/2024/04/23/mac-amx_en/


Loop unrolling
- https://en.wikipedia.org/wiki/Loop_unrolling#:~:text=Loop%20unrolling%2C%20also%20known%20as,or%20by%20an%20optimizing%20compiler.

